In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
from xml.etree import ElementTree
import requests
import time
import pandas as pd
import numpy as np

In [2]:
import matplotlib.image as img 
import matplotlib.pyplot as plt
import pytesseract
import cv2
import os
from skimage import io
from PIL import Image  
import numpy as np
import re
import pandas as pd

In [6]:
path="C:/Users/qjawl/Desktop/대외활동/창의융합캡스톤/cosmetics/cosmetics/images/ocr/ocr_1.jpg"

###################전처리#####################
img = io.imread(path)
#print('원본 이미지 shape :',img.shape)
#plt.imshow(img)
img=cv2.resize(img,None,fx=1.5,fy=1.5) ## 가세연 1.3배
###############################################

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#print('gray 이미지 shape :',gray.shape)
filename = "{}.png".format(os.getpid())
cv2.imwrite(filename, gray)
gray_scale_img=io.imread(filename)
pil_image=Image.fromarray(gray_scale_img)
#pil_image.show()


gray=cv2.threshold(gray,127,255,cv2.THRESH_TRUNC)[1]
filename = "{}.png".format(os.getpid())
cv2.imwrite(filename, gray)
gray_scale_img=io.imread(filename)
pil_image=Image.fromarray(gray_scale_img)
#pil_image.show()


#################################################################333

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe' 
text = pytesseract.image_to_string(Image.open(filename), lang = 'kor')
#print(text)
os.remove(filename)


In [7]:
text_r=text_parse(text)
text_data=cra_preprocessing(text_r)
print(text_data[0])

"정제수"


In [11]:
result=[]
for j in range(len(text_data)):
    p=0
    url="https://kcia.or.kr/cid/search/ingd_list.php?skind=ALL&sword="
    url=url+text_data[j]
    driver = webdriver.Chrome('C:/Users/qjawl/Desktop/chromedriver_win32/chromedriver.exe')
    driver.get(url)

    try:
        driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div/div/table/tbody/tr/td[2]/a').send_keys(Keys.ENTER)
    except:
        result.append('검색결과없음')
        p+=1
    try:
        table=driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div/div/table[1]/tbody')
        trs=table.find_elements_by_tag_name('tr')
        i=0
        for trs_name in trs:
            th=trs_name.find_element_by_tag_name('th')
            if(th.text=="배합목적"):
                i+=1
                break;
            i+=1
        xpath='//*[@id="content"]/div/div[3]/div/div/table/tbody/tr['+str(i)+']/td/p'
        result_text=driver.find_element_by_xpath(xpath)
        print(result_text.text)
        result.append(result_text.text)
    except:
        if(p!=1):
            result.append('검색결과없음')
    driver.quit()

피부컨디셔닝제, 용제
변성제, 착향제, 헤어컨디셔닝제, 보습제, 피부보호제, 피부컨디셔닝제(보습제), 점도감소제
계면활성제(유화제)
유화안정제, 불투명화제, 계면활성제-거품형성제, 점증제-수용성, 점증제-비수용성
착향제, 보존제
착향제, 계면활성제(세정제), 계면활성제(유화제)
착향제, 불투명화제, 계면활성제(세정제), 계면활성제(유화제)
착향제, 보존제, 용제, 점도감소제
기포방지제, 피부보호제, 피부컨디셔닝제(수분차단제), 용제
착향제, 보습제, 피부보습제, 피부컨디셔닝제, 용제, 점도감소제
유화안정제, 점증제-수용성
착향제
모발컨디셔닝제, 피부유연화제
체취방지제, 피부컨디셔닝제(기타)
착향제, 피부컨디셔닝제
착향제
착향제, 피부컨디셔닝제, 용제, 점도감소제
착향제, 피부컨디셔닝제(기타), 피부컨디셔닝제(수분차단제)
착향제, 변색방지제
착향제
감미제, 보습제, 피부컨디셔닝제(보습제), 피부컨디셔닝제(기타)
착향제, 보습제, 피부컨디셔닝제(보습제), 피부컨디셔닝제(기타), pH 조정제
감미제, 착향제
피부컨디셔닝제(기타)
착향제, 피부컨디셔닝제(기타), 피부컨디셔닝제(수분차단제)
착향제, 피부컨디셔닝제(기타)
피부컨디셔닝제(기타)
착향제
착향제
감미제, 착향제, 보습제, 피부컨디셔닝제(보습제)


In [21]:
text_r=pd.Series(text_r)
result=pd.Series(result)
data_result=pd.concat([text_r,result],axis=1)

In [23]:
data_result.to_excel('C:/Users/qjawl/Desktop/창융/화장품데이터/상세데이터.xlsx')

In [5]:
def cra_preprocessing(text_r):
    text_for_cra=[]
    for x in text_r:
        temp=[]
        temp.append("\"")
        for i in x:
            temp.append(i)
        temp.append("\"")
        temp="".join(temp)
        temp=temp.replace(" ","")
        text_for_cra.append(temp)
    return text_for_cra

In [4]:
def filter_data(text_r):
    data_filter_result=[]
    data_filter_label=[]

    for text in text_r:
        sig=0
        label_index=0
        a=0

        for i in range(len(data_base.index)):
            for data in data_base.iloc[i]:
                if data==text:
                    sig=1
                    label_index=i

        if sig ==1:
            data_filter_label.append(data_base.index[label_index])
        else:
            data_filter_label.append('영향없음')
        data_filter_result.append(text)
    data_filter_result=pd.Series(data_filter_result)
    data_filter_label=pd.Series(data_filter_label)
    df_result=pd.concat([data_filter_result,data_filter_label],axis=1)
    return(df_result)

In [3]:
def text_parse(text):
    text_result=[]
    try:
        text=text.replace(" ","")
    except:
        text="".join(text)
        text=text.replace(" ","")
    text=text.replace('\n',"")
    text=text.strip()
    text=list(text)
    text_pre_1=[]
    sig=0
    text_sig=0

    for i in range(len(text)):
        if ((text[i-1]=='성')&(text[i]=='분')):
            sig=1
        if sig==1:
            text_pre_1.append(text[i])
    text_pre_1="".join(text_pre_1)
    #print('성분 Parsing :\n',text_pre_1)
    
    
    p=re.compile('\w+,') #정규식 
    text_pre_2=p.findall(text_pre_1)
    text_pre_2=list(text_pre_2)

    
    for i in range(len(text_pre_2)):
        text_pre_2[i]=text_pre_2[i].replace(',',"") #쉼표 지우기 
        #print(text_pre_2[i])
        text_result.append(text_pre_2[i])
    #print(text_pre_2)
    return(text_result)